In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd, numpy as np 
from travis_attack.config import Config
from travis_attack.insights import get_training_dfs, _prepare_df_concat
from travis_attack.utils import display_all
from IPython.core.debugger import set_trace
import logging 
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
#run_id = 'cdy5d5yu'
run_name = "pleasant-wind-125"
path_run = f"{cfg.path_checkpoints}{run_name}/"

## Looking at some examples 

Below we just look at some examples to get a feel for what is going on. 

In [ ]:
def get_interesting_idx(df, n):
    def get_idx_with_top_column_values(cname, n=5, ascending=False):
        return df[['idx',cname]].\
            drop_duplicates().\
            sort_values(cname, ascending=ascending)\
            ['idx'][0:n].values.tolist()
    
    def sample_idx_with_label_flips(n=5): 
        df1 = df[['idx','label_flip']].query("label_flip!=0")
        if len(df1) == 0 : print("No label flips detected"); return None
        else: return df1.drop_duplicates()['idx'].sample(n).values.tolist()
    
    idx_d = dict(
        random = df.idx.drop_duplicates().sample(n).tolist(),
        label_flips = sample_idx_with_label_flips(n=n),
        idx_n_unique_pp  = get_idx_with_top_column_values('idx_n_unique_pp',n=n,ascending=False),
       # idx_n_pp_changes = get_idx_with_top_column_values('idx_n_pp_changes',n=n,ascending=False),
        high_contradiction = get_idx_with_top_column_values('contradiction_score',n=n,ascending=False)
    )
    return idx_d

def print_stats(df, idx_d, key, i):
    print("\n###############\n")
    print(key, i, "\n")
    if idx_d[key] is None: return
    idx = idx_d[key][i]
    # Setup 
    df1 = df.query('idx==@idx')
    orig = pd.unique(df1['orig_l'])[0]
    print("Original:", orig)
    print("Original label", pd.unique(df1['orig_label'])[0] )
    pp_all = list(df1['pp_l'])
    #print("All paraphrases", pp_all)
    pp_unique = list(pd.unique(df1['pp_l']))
    n_pp_unique = len(pp_unique)

    # showing a "timeline" of how the paraphrases change over the epochs
    g_fields = ["pp_l","pp_truelabel_probs","vm_score","sts_score","pp_letter_diff", "contradiction_score", "reward","label_flip", 'pp_logp',
       'ref_logp', 'kl_div', 'reward_with_kl', 'loss']
    #g_fields = ["pp_l","vm_score"]
    g = df1.groupby(g_fields).agg({'epoch' : lambda x: list(x)})
    g = g.sort_values(by='epoch', key = lambda col: col.map(lambda x: np.min(x)))
    print("Unique paraphrases:", n_pp_unique)
    print("How the paraphrases change:")
    display_all(g)

    # Showing a dataframe of the few best paraphrases
    best_pps = df1.sort_values('reward', ascending=False).iloc[0]
    print("Best Paraphrase")
    display_all(best_pps.to_frame().T)
        
def print_interesting_text_stats(df, n): 
    idx_d = get_interesting_idx(df, n)
    for key in idx_d.keys():
        for i in range(n): 
            print_stats(df, idx_d, key,i)

In [ ]:
df_d['training_step'].columns

Index(['idx', 'epoch', 'orig_l', 'pp_l', 'orig_truelabel_probs',
       'pp_truelabel_probs', 'pp_predclass_probs', 'orig_label',
       'pp_predclass', 'label_flip', 'vm_score', 'sts_score', 'pp_letter_diff',
       'pp_letter_percent', 'contradiction_score', 'reward', 'pp_logp',
       'ref_logp', 'kl_div', 'reward_with_kl', 'loss', 'batch_num',
       'global_step', 'acc_num', 'loss_sum', 'loss_batch',
       'label_flip_fraction', 'orig_length', 'orig_batch_size', 'pp_length',
       'pp_batch_size', 'time_generate_pp', 'time_loss_fn', 'time_reward_fn',
       'time_vm_scores', 'time_sts_scores', 'time_pp_letter_diff',
       'time_contradiction_scores', 'time_pp_logp', 'time_log_entropy',
       'time_log_token_probabilities', 'time_ref_logprobs',
       'time_loss_fn_loss_calc', 'time_backwards', 'time_calc_gradient_norm',
       'time_opt_step_and_calc_param_norm', 'time_opt_step', 'idx_n_unique_pp',
       'idx_n_pp_changes', 'epoch_of_first_label_flip', 'n_words_orig',
       

In [ ]:
split = 'training_step'
df_d = get_training_dfs(path_run, postprocessed=True)
idx_d = get_interesting_idx(df_d[split], n=5)
print_interesting_text_stats(df_d[split], n=5)


###############

random 0 

Original: anyone not into high-tech splatterfests is advised to take the warning literally , and log on to something more user-friendly .
Original label 0
Unique paraphrases: 5
How the paraphrases change:


,,,,,,,,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,pp_letter_diff,contradiction_score,reward,label_flip,pp_logp,ref_logp,kl_div,reward_with_kl,loss,
The warning is meant to encourage everyone to log on to a better place.,0.32892,0.34492,0.57246,56,0.002114,0.0,1,-20.570507,-73.204269,52.633762,0.0,-0.0,[1]
Everyone is advised.,0.31579,0.35805,0.24189,107,0.170527,0.0,1,-4.945487,-28.974981,24.029495,0.0,-0.0,[2]
"Also, if that.",0.55794,0.11590,0.23023,113,0.009639,0.0,0,-12.836550,-23.372009,10.535460,0.0,-0.0,[3]
It is.. A.,0.33246,0.34138,0.15107,117,0.205445,0.0,1,-11.306890,-28.832207,17.525316,0.0,-0.0,[4]
It is..,0.34807,0.32578,0.16802,120,0.031490,0.0,1,-1.970884,-16.545067,14.574183,0.0,-0.0,[5]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,ref_logp,kl_div,reward_with_kl,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
33440,7669,1,"anyone not into high-tech splatterfests is advised to take the warning literally , and log on to something more user-friendly .",The warning is meant to encourage everyone to log on to a better place.,0.673842,0.32892,0.671081,0,1,1,0.34492,0.57246,56,0.559055,0.00211434,0,-20.5705,-73.2043,52.6338,0,-0,362,362,0,0,0,0.25,32,8,21,8,0.345518,0.717142,0.0525293,0.00814862,0.0273254,0.000119647,0.0161784,0.0144224,0.00360222,0.00534629,0.64977,0.000254783,0.479942,0.0548664,3.13013e-05,1.94786e-06,5,4,1,19,1,4,0,103,14,1,1,0,57,5,0,3,0,46,"[2, 3, 4, 5, 6, 7, 8, 10, 16, 17, 18, 19, 20, 21, 28, 29, 30, 31, 32]","[anyone not into high- tech splatterfests, advised, take the warning literally, and, something more user- friendly]","[0, 1, 11, 13, 14, 15, 25, 26, 27]","[The warning, meant, encourage everyone to, a better place]","[9, 12, 22, 23, 24, 33]","[is, to, log on to, .]",4,4,9,19,False,False,False,19



###############

random 1 

Original: seagal ran out of movies years ago , and this is just the proof .
Original label 0
Unique paraphrases: 5
How the paraphrases change:


...

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,ref_logp,kl_div,reward_with_kl,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
32295,7404,1,i hate the feeling of having been slimed in the name of high art .,I didn' being slim,0.865071,0.75067,0.750669,0,0,0,0.1144,0.48204,48,0.272727,0.963414,0,-8.74972,-39.4815,30.7318,0,-0,716,716,0,-4.58405,-0.286503,0.125,24,8,13,8,0.207462,0.432661,0.0491012,0.00726504,0.0254721,8.4104e-05,0.0155804,0.0118924,0.00245932,0.00640699,0.371267,0.00023691,0.296563,0.0475441,3.18349e-05,2.25939e-06,5,4,2,14,1,1,0,51,4,1,1,0,14,10,0,0,0,37,"[4, 5, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19, 20]","[i hate the feeling of having been slimed, in the name of high art.]","[0, 1, 2, 3, 13]","[I didn' being, slim]",[],[],2,2,5,15,False,False,False,15



###############

high_contradiction 4 

Original: beneath clouds is a succinct low-budget film whose compelling characters and intelligent script are exactly what was missing from rabbit-proof fence .
Original label 1
Unique paraphrases: 5
How the paraphrases change:


,,,,,,,,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,pp_letter_diff,contradiction_score,reward,label_flip,pp_logp,ref_logp,kl_div,reward_with_kl,loss,
The film's briefness and lack of depth make it very well suited for small screening halls.,0.13357,0.78590,0.59095,60,0.029497,0.0,1,-41.578568,-97.544090,55.965523,0.0,-0.0,[1]
the was..,0.35307,0.56640,0.18755,141,0.594505,0.0,1,-7.506480,-30.012928,22.506447,0.0,-0.0,[2]
It's a dull and boring.,0.06185,0.85762,0.18666,127,0.999587,0.0,1,-13.716248,-35.283470,21.567223,0.0,-0.0,[3]
"., at one, also,!",0.54277,0.37670,0.16717,133,0.046067,0.0,0,-19.467342,-44.574295,25.106953,0.0,-0.0,[4]
The.. A.,0.59156,0.32791,0.18827,142,0.836482,0.0,0,-5.484054,-24.918808,19.434753,0.0,-0.0,[5]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,ref_logp,kl_div,reward_with_kl,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
5775,1306,1,beneath clouds is a succinct low-budget film whose compelling characters and intelligent script are exactly what was missing from rabbit-proof fence .,The film's briefness and lack of depth make it very well suited for small screening halls.,0.919471,0.13357,0.866428,1,0,1,0.7859,0.59095,60,0.6,0.0294972,0,-41.5786,-97.5441,55.9655,0,-0,438,438,0,-45.9049,-2.86906,0.5,32,8,25,8,0.378253,0.852264,0.0509828,0.00753365,0.0267984,8.12029e-05,0.0158493,0.0145136,0.00398528,0.0056928,0.78633,0.000254625,0.579283,0.0517453,3.12449e-05,2.10106e-06,5,4,1,21,1,3,0,126,17,1,2,0,73,4,0,1,0,53,"[1, 2, 3, 4, 5, 6, 7, 8, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]","[beneath clouds is a succinct low- budget, whose compelling characters, intelligent script are exactly what was missing from rabbit- proof fence]","[0, 10, 11, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]","[The, 's briefness, lack of depth make it very well suited for small screening halls]","[9, 15, 40]","[film, and, .]",3,3,15,23,False,False,False,23


## Looking at common removals and insertions 

In [ ]:
def get_common_removals_and_insertions(df_concat): 
    idx = df_concat[['data_split','orig_l', 'pp_l']].drop_duplicates().index
    df_unique_pp = df_concat[['data_split','orig_l', 'pp_l','insertions', 'removals']].iloc[idx]
    def flatten_list(l): return [item for sublist in l for item in sublist] 
    removals_flat   =  flatten_list(df_unique_pp['removals'].values)
    insertions_flat =  flatten_list(df_unique_pp['insertions'].values)
    return pd.value_counts(removals_flat), pd.value_counts(insertions_flat)

In [ ]:
df_concat = _prepare_df_concat(df_d)
removals, insertions = get_common_removals_and_insertions(df_concat)

print("\n#### REMOVALS ####\n")
print(removals.head(30))
print("\n#### INSERTIONS ####\n")
print(insertions.head(30))

Here you can look at a specific phrase and examples of where it appears. 

In [ ]:
def investigate_phrase(phrase, cname, n ): 
    mask = [phrase in strs for strs in df_concat[cname]]
    display_all(df_concat[mask].sample(n))

In [ ]:
investigate_phrase('despite', 'removals', 4)